<h1>Fusion Choropleth maps</h1>
In this assignment we'll examine three different ways of looking at the Covid-19 case data in New York State. All three maps use the New York state counties map as the base map. While we're going to create a few very nice maps, we'll also get a lot of Pandas practice along the way! The three different maps we'll create are:

<span style="color:blue">Problem 1:</span> The first map looks at the total number of positive cases in each county adjusted for population (incidence). The resulting choropleth map will show which counties were the hardest hit by Covid-19 from the start of the pandemic till the last data date. My version of this map (last data date = 09/18/2023) is in the file <span style="color:blue">Problem 1.html</span>

<span style="color:blue">Problem 2:</span> The second map constructs a <a href="https://github.com/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb">time slider choropleth map</a> using the same data as in problem 1 with the difference that the output is a map for each day between March 1st 2020 and the latest data date. To smooth out noise, rather than plotting the raw incidence for each day, we'll plot the 8 day moving average of the daily incidence (incidence = cases/population). My version of this map (last data date = 09/18/2023) is in the file <span style="color:blue">Problem 2.html</span>

<span style="color:blue">Problem 3:</span> If you slide through the map from Problem 2, you'll notice that there are large chunks of time where the entire map is almost yellow. This is because the choropleth map is using a range from the lowest 8 day moving average to the highest 8 day moving average to figure out how much a county should be shaded. Because  the omicron spike of late December 2021 and early January 2022, all other periods look insignificant in comparison. An alternative way at looking at the data would be to construct daily choropleth maps that focused on the relative incidence of Covid-19 across counties on any given day. One way to do this is to rank the counties by the incidence levels for each day separately. In the third choropleth map, we'll construct a time slider choropleth map which uses the 8 day moving average of these daily ranks (highest to lowest). Public health officials will find this more useful than the problem 2 map because they can move resources (testing kits, hospital supplies, treatments, etc.) to the counties with a relatively higher incidence even when overall cases are quite low. My version of this map (last data date = 09/18/2023) is in the file <span style="color:blue">Problem 3.html</span> 


<h2>Getting the data</h2>
<h3>Data sources</h3>
<li>List of New York counties (file: NewYork_DemographicsByCounty_sample.csv)</li>
<li>Population of counties - 2020 census (same source as above)</li>
<li>GeoJson file with county boundaries (File cugir-007865-geojson.json)</li>
<li>Covid cases (date, county, new positive cases) by county (file: testing_data.csv)</li>

<h3>Data set up</h3>
<p>
<span style="color:green;font-size:20px">Create a population_and_county_df</span>
<li>Read populations and counties into a dataframe population_and_county_df (use <a href="https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html">pd.read_csv</a>)</li>
    <li>Note that the population numbers use commas as thousands separators. Check the documentation of read_csv to see how to handle those</li>
<li>Remove the "County" from each county name (this will be necessary later)</li>
<li>Drop any rows that are not useful (there should be 62 counties in total)</li>
<li>population_and_county_df should look something like:</li>
<pre>
	County	Population
0	Albany	315811
1	Allegany	46694
2	Bronx	1379946
3	Broome	197117
4	Cattaraugus	76439
...	...	...
57	Washington	60841
58	Wayne	91125
59	Westchester	990427
60	Wyoming	39666
61	Yates	24451
</pre>

In [ ]:
import pandas as pd
import numpy as np
import datetime

import pandas as pd
#Use pd.read_csv to read the file
population_and_county_df = 

#Remove County from county names (e.g., New York County should become New York)
population_and_county_df['County']=


<span style="color:green;font-size:20px">Read the geojson file into a python json object</span>
<li>Store this in a variable geojson_data</li>
<li>We can use the file directly for problem 1 but will need to store it in in a variable and modify it for Problem 2 </li>


In [ ]:
#Geojson data
import json
county_geojson_file = "cugir-007865-geojson.json"
#open the file and read it

#then convert json into python
geojson_data = 





<span style="color:green;font-size:20px">get all covid-19 data from testing_data.csv into the variable df using pandas read_csv function </span>

<span style="color:green;font-size:20px">Join population_and_county_df to the cases dataframe</span>
<li>Use pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html">join</a> function to do the join</li>
<li>This will be the base dataset for both problems</li>
<li>You may need to use pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html">pd.set_index</a> to get the join to work (depends on how you do it). Either way, you're going to need to work with set_index and reset_index quite a bit so take a look at the documentation and a few examples</li>
<li>A sample of what df should contain</li>
<pre>
	Test Date	Positives	Population
County			
Albany	09/18/2023	17	315811
Albany	09/17/2023	21	315811
Albany	09/16/2023	13	315811
Albany	09/15/2023	31	315811
Albany	09/14/2023	27	315811
...	...	...	...
Yates	03/05/2020	0	24451
Yates	03/04/2020	0	24451
Yates	03/03/2020	0	24451
Yates	03/02/2020	0	24451
Yates	03/01/2020	0	24451
80414 rows × 3 columns


</pre>


In [ ]:
#Read testing_data.csv using pd.read_csv
df = 

#Set the index to County and then join with population_and_county_df
#Also drop NaNs (see pd.dropna)
df = 

df.info()

<h1>Problem 1: Incidence choropleth map</h1>
<li>For this problem, you need to create a choropleth map of NY counties that are colored by the incidence of covid-19 in the county</li>
<li>We'll define <span style="color:blue">incidence</span> as the cumulative number of cases divided by the population</li>
<li>The template for the map is in the cell below. You need to create a new column <span style="color:blue">incidence</span> and fill in the missing attributes in the template</li>
<li>group the data by county and using the sum aggregate function, total the number of cases for each county. Divide this by the population for the county (no for loops please!). 
    <li>You'll need to get the population from population_and_county_df. Align the indexes using set_index and then join the two dataframes. Then divide the total cases by the population</li>
    <li>This should result in a dataframe with the incidence for each county</li>
<li>Choose an appopriate custom color scheme from <a href="https://github.com/python-visualization/folium/blob/v0.2.0/folium/utilities.py#L104">https://github.com/python-visualization/folium/blob/v0.2.0/folium/utilities.py#L104</a></li>
<li>The dataframe for folium (note that your column ordering may be different):</li>
<pre>
	Positives	incidence	Population
County			
Albany	86740	0.274658	315811
Allegany	11904	0.254936	46694
Bronx	582518	0.422131	1379946
Broome	67783	0.343872	197117
Cattaraugus	21318	0.278889	76439
...	...	...	...
Washington	17756	0.291843	60841
Wayne	24851	0.272713	91125
Westchester	374445	0.378064	990427
Wyoming	10854	0.273635	39666
Yates	4965	0.203059	24451
62 rows × 3 columns

In [ ]:
#Group df by County
groups = 

#Create a sum_df that contains the grouped by County sum of Positives in a column called
#incidence
#sum_df should only contain incidence (with County as the key)
#use pd.drop if you need to drop columns
sum_df = 

#Join sum_df and poplulation_and_county_df so that you now have a Population column
folium_df = 

#divide the incidence column by the population column
folium_df.incidence = 

#Create the map object
import folium
m = 

#Create the Choropleth map and add it to m
#Note that you will need to link county names in folium_df with feature.properties.name
#in the geojson file
c = 

#Add a tooltip so that the county name pops up when you hover over it
c.geojson.add_child(
    
)

#See the map on your Jupyter notebook
m

#If all looks good, uncomment the next line to save it to a file
#m.save("Problem 1.html")

<h1>Problem 2: Create a time slider choropleth map</h1>
<li>In this problem, you'll create a choropleth map object that changes with changes in the date. The map will include a slider that the user can use to "slide" through the dates (from March 1st 2020 to the last data date)</li>
<li>The thing to focus on here is the time slider. Time, as we know (or should know anyway) is an inexorable thing that marches on regardless of our efforts to contain it. What our slider needs is an understanding of where the data points are in that march of time</li>
<li>The way we'll deal with it here is to use the concept of <a href="https://en.wikipedia.org/wiki/Unix_time">Unix Time</a>. Unix time is stored as an integer starting with 0 at 00:00:00 of 1st January 1970 (think of that date as the unix big bang date) and then adds 1 to it for each second. </li>
<li>See below for what today is in unix time</li>
<li>(More on time later) A time slider choropleth map is created using the TimeSliderChoropleth object (see <a href="https://python-visualization.github.io/folium/latest/user_guide/plugins/timeslider_choropleth.html">https://python-visualization.github.io/folium/latest/user_guide/plugins/timeslider_choropleth.html</a>. You can download the notebook and play around with it)</li>

<span style="color:green;font-size:30px">Rough steps</span>
<p></p>
<span style="color:green;font-size:24px">Calculate 8 day moving average</span>
<li>calculate the incidence (new positives/population) for each row in df. This should be straightforward since both are columns of the df </li>
<li>Smooth out incidence by computing an 8-day moving average. Note that each county will have its own moving average series (i.e., you can't construct rolling windows on the entire incidence column but must first group the data by county</li>
<ol>
    <li>use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html">pd.groupby</a> to group the data by county and then apply the
<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html">groupby.transform</a> to apply a function on each group individually</li>
    <li>the function should construct a rolling mean (call <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html">rolling</a> and <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html">mean</a> in the function. Also, fill nans with zeros using <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html">fillna</a>). Use the moving average examples we constructed in class as a guide</li>
</ol>
<p></p>
<span style="color:green;font-size:24px">Convert test date to unix time</span>
<li>I've done this for you but do walk through the code</li>
<li>Convert test_date to pandas datetime</li>
<li>Add one day to each value</li>
<li>the function astype(int) returns the integer time in nanoseconds given a datetime object</li>
<li>The result is in nanoseconds, we'll divide this by 10 to the power of 9 to get seconds (unix time)</li>
<li><b>Note:</b> Why add one day? When we do the division by 10**9, we don't want to end up with a decimal point in the result. Therefore we use the integer division (//) operator. Integer division truncates the result and, since all our times are exactly at 00:00:00.0, this ends up pushing the date to the previous day in 99.99% of the cases. There may be a case where the division was exact, in which case we'll get the wrong date, but the odds are very low and, anyway, nobody said life is perfect!</li> 

<p></p>
<span style="color:green;font-size:24px">Make a colormap</span> 
<li>We need to assign a color and opacity to each county at each point in time based on the value of the moving average of incidence</li>
<li>For opacity, we'll use a constant. 0.3 is probably good enough</li>
<li>For color, we'll choose a color palette (YlOrRd is my choice but choose any you like from the palette link in problem 1 above)</li>
<li>Assign the lowest color (yellow with my choice) to the lowest value in the ma8 column</li>
<li>Assign the highest color (red with my choice) to the highest value in the ma8 column</li>
<li><span style="color:blue">branca</span> is a companion library to folium that contains non geography specific map information (like colors!). We'll use that to create a linear color scale from the lowest ma8 value to the highest ma8 value</li>
<li>Then add two columns to df. The column <b>color</b> with the color (each ma8 value will map to a specific color) and the <b>opacity</b> with the opacity (a constant e.g., 0.3)</li>

<p></p>
<span style="color:green;font-size:24px">Replace county names</span>
<li>Since the geojson file needs ids to be strings with no special characters, and we have strings like "New York" and "St. Lawrence" in the data (with a space and a period), we'll replace all county names with "0", "1", "2", ... in the dataframe</li>
<li>We'll have to do this in the geojson object as well (currently, those are even more obscure), so we'll create a dictionary that maps county names to integer strings</li>
<li>Example:</li>
<pre>
{'Kings': '0',
 'Queens': '1',
 'New York': '2',
 'Suffolk': '3',
 'Bronx': '4',
 'Nassau': '5',
 .....
}
</pre>
<li>Make a list of county names (you can get this from population_and_county_df</li>
<li>Create an empty dictionary (county_mapping)</li>
<li>Iterate through the county names list adding key (county name) and value (county number as str) to the dict</li>
<li>Replace county names in df with county numbers (use apply for this). You might want to keep a separate column for county names, though that's not really necessary</li>

<p></p>
<span style="color:green;font-size:24px">Create a sytle dictionary</span>
<li><b>Note</b>: At various points, you may end up with data in the index that you want to use as if it was in a column. However, a dataframe index is not a column so pandas provides a handy way of converting the index into a dataframe column. <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html">reset_index</a>. For moving a column to an index, use <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html">set_index</a></li>
<li>The style dictionary contains the elements that go into the map (the test_date - a unix time value; the color - a color that reflects the ma8 value; and the opacity - a constant)</li>
<li>iterate through the counties
    <ol>
        <li>create a df2 for each county that contains the test date, color and opacity for each test date. Use set_index and make test_date the index</li>
        <li>convert the df2 into a dictionary using <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html">df.to_dict()</a>. Use the orient="index" option for this conversion</li>
        <li>add (county id, df2 as a dictionary) as a key value pair to style_dict (make sure your style_dict looks exactly like mine below!</li>
    </ol>
</li>
<li>An example of df2:</li>
<pre>
	color	opacity
unix_date		
1695081600	#ffffccff	0.5
1694995200	#ffffccff	0.5
1694908800	#ffffccff	0.5
1694822400	#ffffccff	0.5
1694736000	#ffffccff	0.5
...	...	...
1583452800	#ffffccff	0.5
1583366400	#ffffccff	0.5
1583280000	#ffffccff	0.5
1583193600	#ffffccff	0.5
1583107200	#ffffccff	0.5
1297 rows × 2 columns
</pre>

<p></p>
And styledict:
<pre>
{'0': {1695081600: {'color': '#ffffccff', 'opacity': 0.5},
  1694995200: {'color': '#ffffccff', 'opacity': 0.5},
  1694908800: {'color': '#ffffccff', 'opacity': 0.5},
  1694822400: {'color': '#ffffccff', 'opacity': 0.5},
  1694736000: {'color': '#ffffccff', 'opacity': 0.5},
  1694649600: {'color': '#ffffccff', 'opacity': 0.5},
  1694563200: {'color': '#ffffccff', 'opacity': 0.5},
  1694476800: {'color': '#fffec9ff', 'opacity': 0.5},
  1694390400: {'color': '#fffec9ff', 'opacity': 0.5},
</pre>
<p>
The structure of styledict:
<pre>
{county_number : {unix_date1: {color: color_value,opacity: opacity_value},
               : {unix_date2: {color: color_value,opacity: opacity_value},
    ....
{county_number : {unix_date1: {color: color_value,opacity: opacity_value},
               : {unix_date2: {color: color_value,opacity: opacity_value},
    ....
</pre>    

<p></p>
<span style="color:green;font-size:24px">Create the time slider choropleth map</span>
<li>I've done this part for you</li>
<li>Because we're anchoring the data to the maximum incidence since March 2020, you'll notice that the map discriminates between counties the most during the first omicron wave (the number of identified cases was an order of magnitude higher than at other times during the pandemic). We'll, sort of, address this issue in the next problem.</li>

In [4]:
#IMPORTANT STEP. This will save the current version of your df in df.save
#If you mess up, you won't have to run problem 1 again
df.info()
df_save = df.copy()

<class 'pandas.core.frame.DataFrame'>
Index: 80414 entries, Albany to Yates
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Test Date   80414 non-null  object
 1   Positives   80414 non-null  int64 
 2   Population  80414 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ MB


In [ ]:
#Copy the dataframe from df_save and remove the index
df = df_save.copy().reset_index()

#Create a list of counties from population_and_county_df
#(use unique to make sure you don't have duplicates)
counties = 

#Get the incidence of covid for each date
#(Divide Positives column by Population column)
df['incidence'] = 

#Get the 8 day moving average
df['ma8'] = 

#The following statement converts the date from its yyyy-mm-ddThours:minutes:seconds format to unix time
#There are many ways to do this (ask google or chatGPT) but I've done this for you
df['unix_date'] = (pd.to_datetime(df['Test Date']) + pd.Timedelta(1,'d')).view(int) //(10 ** 9)

#branca is a folium adjunct that contains functionality for non map specific features
#documentation at https://python-visualization.github.io/branca/
#We'll use it to scale values of the 8 day moving average between the highest value 
# and the lowest value
#Already done but you can change YlOrRd if you don't like the colors

import branca.colormap as cm
max_value = df["ma8"].max()
min_value = df['ma8'].min()
cmap = cm.linear.YlOrRd_09.scale(min_value, max_value)
df['color'] = df["ma8"].map(cmap)
df['opacity']=0.5

#Create the mapping object.
#Map each county to a unique value 0,1,2, etc.
#Make sure the 0, 1, 2 etc are str objects not ints
#e.g., "Albany" : "0", ...

county_mapping = dict()
#Code for adding the mapping goes here

#Create a new column that contains the county numbers (use pd.apply on County column)
df['County_Number'] = 


#Create the styledict
#Easy to mess this up. Make sure you have exactly the same structure as described above

#Initialize an empty styledict
styledict = dict()

#Set the index of df to County_number


#Iterate through each unique county_number (i.e., 0 to 61)
for county in df.index.unique():
    #Create a df2 from df for this county number
    #Columns should be unix_date, color, opacity
    #unix_date should be the index 
    df2 = 
    
    #Add a new key (this county number) to styledict with df2 converted
    # to a dictionary (use orient="index")
    styledict[county] = 
    
#Add a new attribute to properties in the geojson_data object
#Name this attribute "id" (it cannot have any other name!)
#The value of this attribute is the county number that corresponds to the county name
#(county name is in feature.properties.name)


#Finally, create a TimeSliderChoropleth

import folium,json
from folium.plugins import TimeSliderChoropleth
m = 
g = TimeSliderChoropleth(

).add_to(m)

m

#If it looks good, uncomment the next line
#m.save("Problem 2.html")

<h1>Problem 3: Get daily covid incidence ranks and create a timeslider choropleth map</h1>
In problem 2, we saw how the covid case incidence changed over time for each county in New York State. For resource allocation, however, a administrator may want to know which counties have a higher incidence <span style="color:red">relative</span> to other counties. The administrator could then move resources (testing, doctors and nurses, hospital supplies) to the counties that have a relatively higher number of cases. 

For this problem, we'll create a time slider choropleth map that, at each point in time, shades the counties with a higher relative incidence darker than the counties with a lower relative incidence. To get relative incidence, we'll rank the counties by their incidence (cases/population) at each point in time. To smooth out noise, we'll use the 8 day moving average of the rank as the data for the choropleth map.

<span style="color:green;font-size:30px">Rough steps</span>
<p></p>
<span style="color:green;font-size:24px">Calculate relative incidences by date</span>
<li>For each date, sum up the values in the incidence column generated in problem 2</li>
<li>Store this in the variable <span style="color:blue">inc_sum_df</span></li>
<li>Then divide each incidence in df by the total incidence for the same date to get what proportion of population adjusted cases are in a county. Write a function to do this so that you can set any divide by zero values at zero. inc_sum_df will give the total incidence for a given date (the denominator)</li>
<li>Then use apply to create a new dataframe column "relative_incidence"</li>
<li>A sample of what to expect in inc_sum_df:</li>
<pre>
	incidence
unix_date	
1583107200	0.000001
1583193600	0.000003
1583280000	0.000012
1583366400	0.000021
1583452800	0.000015
...	...
1694736000	0.008509
1694822400	0.007105
1694908800	0.005176
1694995200	0.005618
1695081600	0.011308
1297 rows × 1 columns
</pre>

<li>A sample of what to expect in df['relative_incidence']. The NaNs come from dividing very small numbers by other very small numbers (try..except won't catch those but you can ignore them)</li>
<pre>
County_Number
0     0.476035
0     1.183627
0     0.795310
0     1.381558
0     1.004786
        ...   
61    0.000000
61    0.000000
61    0.000000
61    0.000000
61    0.000000
Name: relative_incidence, Length: 80414, dtype: float64
</pre>

<p></p>
<span style="color:green;font-size:24px">Calculate ranks and moving averages of rank</span>
<li>Create a column, rank, that ranks the relative incidence for each county within each date. Use groupby and <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rank.html">df.rank</a> to do this</li>
<li>For each county, calculate the 8 day moving average of rank. Use groupby, rolling, and mean and fill any NaNs with 0.0 (see <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html">df.fillna</a>). Store this in a column named rma8</li>
<li>Example of df['rank']:</li>
<pre>
County_Number
0      4.0
0     28.0
0     14.0
0     33.0
0     14.0
      ... 
61     1.0
61     1.0
61     1.0
61     1.0
61     1.0
Name: rank, Length: 80414, dtype: float64
</pre>
<li>Example of df['rma8'] (for test date 01/06/2022):</li>
<pre>
County_Number
0     34.750
1      3.750
2     55.750
3     26.125
4     14.750
       ...  
57    31.125
58    21.125
59    37.125
60    30.500
61     7.250
Name: rma8, Length: 62, dtype: float64
</pre>

<p></p>
<span style="color:green;font-size:24px">Create the time slider choropleth map</span>
<li>First use branca to generate the color and the opacity. Note that you can use a global ma8 max and min because the ranks for each day are scaled to the number of counties and the global max and min will, therefore, be almost the same as the max and the min for each day. This is a lot easier than scaling each day separately</li>
<li>Draw the map. This is identical to the map in problem 2</li>



In [ ]:
#Use save2 to start with a clean copy of df. Get rid of the index
df = df_save2.copy().reset_index()

#inc_sum_df 
# sort df by unix_date, then groupby unix_date, extract County and incidence and
# sum the result 
#This should give you the sum of the incidence across all counties for each date

inc_sum_df = 

#Now calculate the pct_incidence
#divide each incidence in df by the sum for that date (from inc_sum_df)
#because we have zeros, we need to write a function to handle the zero division cases
#if zerodivisionerror, then return 0.0
def pct_incidence(x):

    
#Now apply the function to the df and create a new "relative_incidence" column
df['relative_incidence']=

#Calculate the rank within a date for each relative_incidence
#For this, goup by unix_date and then get the rank
#Use the dense parameter for rank - that way each county will have a distinct rank
df['rank'] = 


#Calculate the 8 day moving average of rank. Use transform as in Problem 2
df['rma8'] = 

#The colormap using rma8 (done for you)
import branca.colormap as cm
max_value = df["rma8"].max()
min_value = df['rma8'].min()
cmap = cm.linear.YlOrRd_09.scale(min_value, max_value)
df['color'] = df["rma8"].map(cmap)
df['opacity']=0.5


#Generate the styledict (same as in problem 2)
styledict = dict()

#Update the geojson file (change counties to "0", "1", "2", ....)
#Hopefully already done for problem 2


#Create the time slider choropleth map

import folium,json
from folium.plugins import TimeSliderChoropleth
m = 

g = TimeSliderChoropleth(

).add_to(m)

m

#If all looks good, uncomment the next line
m.save("Problem 3.html")